Calls C program with parameters that are given in the arrays

In [1]:
from subprocess import call, check_output
import subprocess
import numpy as np

NNodes = [1600]
phifloat = [0.005, 0.15, 0.9] #list(np.linspace(0.01, 1, num=100))
MAX_ITER = [100000000]
ITER_step = [500]
filename = "try"
redundancy = 10000
manNodes = list(np.arange(0,10,1))
manip = 1
manipMode = -1

Now with multiprocessing:

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import itertools
from math import *
from random import randint


prog = 0

results2 = []
#As above just with a filename dependent on the worker to avoid corruptions. Will create a file for each thread
def RunCProgram(NN, phi, M_Iter, step, manNodes):
    global filename
    nfilename = filename + str( multiprocessing.current_process()._identity[0])
    
    if len(nfilename)>15:
        nfilename = filename + str(print(randint(0, 1000)))
    
    global prog
    for x in range(redundancy):
        try:
            check_output(["./a.out", str(NN), str(NN*2), str(NN/10), str(phi), str(M_Iter), str(step), nfilename, str(manip), str(manNodes), str(manipMode)])
            prog +=1 
            if  (prog/comb)%10 == 0 and prog/comb < 100:
                print(comb/prog)
        except subprocess.CalledProcessError as e:
            raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    return


if __name__ == '__main__':
    NThreads = multiprocessing.cpu_count()
    param = []
    param.append(NNodes)
    param.append(phifloat)
    param.append(MAX_ITER)
    param.append(ITER_step)
    param.append(manNodes)
    InputArray = list(itertools.product(*param))
    comb = len(InputArray)*redundancy
    with Pool(processes=NThreads) as pool:         # start 4 worker processes
        pool.starmap(RunCProgram, InputArray)       # prints "[0, 1, 4,..., 81]"